# Imports

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches,patheffects

import torch
from torchvision import datasets,transforms,models
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader , Dataset
import torchvision.transforms.functional as FT

from PIL import Image
import xml.etree.ElementTree as ET


from pathlib import Path
import json
import os
import random
from tqdm import tqdm,trange
import time
from math import sqrt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"{torch.__version__} and {device}")

1.0.1.post2 and cpu


In [0]:
def xy_to_cxcy(xy):
    """
    Convert bounding boxes from boundary coordinates (x_min, y_min, x_max, y_max) to center-size coordinates (c_x, c_y, w, h).

    :param xy: bounding boxes in boundary coordinates, a tensor of size (n_boxes, 4)
    :return: bounding boxes in center-size coordinates, a tensor of size (n_boxes, 4)
    """
    return torch.cat([(xy[:, 2:] + xy[:, :2]) / 2,  # c_x=(x_max+x_min)/2, similarly for c_y = (y_max+y_min)/2
                      xy[:, 2:] - xy[:, :2]], 1)  # w = x_max-x_min, similarly for h


def cxcy_to_xy(cxcy):
    """
    Convert bounding boxes from center-size coordinates (c_x, c_y, w, h) to boundary coordinates (x_min, y_min, x_max, y_max).

    :param cxcy: bounding boxes in center-size coordinates, a tensor of size (n_boxes, 4)
    :return: bounding boxes in boundary coordinates, a tensor of size (n_boxes, 4)
    """
    return torch.cat([cxcy[:, :2] - (cxcy[:, 2:] / 2),  # c_x-w=x_min, c_y-h=y_min
                      cxcy[:, :2] + (cxcy[:, 2:] / 2)], 1)  # x_max, y_max


def cxcy_to_gcxgcy(cxcy, priors_cxcy):
    """
    Encode bounding boxes (that are in center-size form) w.r.t. the corresponding prior boxes (that are in center-size form).

    For the center coordinates, find the offset with respect to the prior box, and scale by the size of the prior box.
    For the size coordinates, scale by the size of the prior box, and convert to the log-space.

    In the model, we are predicting bounding box coordinates in this encoded form.

    :param cxcy: bounding boxes in center-size coordinates, a tensor of size (n_priors, 4)
    :param priors_cxcy: prior boxes with respect to which the encoding must be performed, a tensor of size (n_priors, 4)
    :return: encoded bounding boxes, a tensor of size (n_priors, 4)
    """

    # The 10 and 5 below are referred to as 'variances' in the original Caffe repo, completely empirical
    # They are for some sort of numerical conditioning, for 'scaling the localization gradient'
    # See https://github.com/weiliu89/caffe/issues/155
    return torch.cat([(cxcy[:, :2] - priors_cxcy[:, :2]) / (priors_cxcy[:, 2:] / 10),  # g_c_x, g_c_y
                      torch.log(cxcy[:, 2:] / priors_cxcy[:, 2:]) * 5], 1)  # g_w, g_h


def gcxgcy_to_cxcy(gcxgcy, priors_cxcy):
    """
    Decode bounding box coordinates predicted by the model, since they are encoded in the form mentioned above.

    They are decoded into center-size coordinates.

    This is the inverse of the function above.

    :param gcxgcy: encoded bounding boxes, i.e. output of the model, a tensor of size (n_priors, 4)
    :param priors_cxcy: prior boxes with respect to which the encoding is defined, a tensor of size (n_priors, 4)
    :return: decoded bounding boxes in center-size form, a tensor of size (n_priors, 4)
    """

    return torch.cat([gcxgcy[:, :2] * priors_cxcy[:, 2:] / 10 + priors_cxcy[:, :2],  # c_x, c_y
                      torch.exp(gcxgcy[:, 2:] / 5) * priors_cxcy[:, 2:]], 1)  # w, h

In [58]:
s1=[[0,0,3,4],[0,0,3,4]]
set1=torch.tensor(s1,dtype=torch.float)
cxcy=xy_to_cxcy(set1)
print("Cx_C_y",cxcy)
gcxgcy=cxcy_to_gcxgcy(cxcy,set1)
print("G_cx_G_cy",gcxgcy)

Cx_C_y tensor([[1.5000, 2.0000, 3.0000, 4.0000],
        [1.5000, 2.0000, 3.0000, 4.0000]])
G_cx_G_cy tensor([[5., 5., 0., 0.],
        [5., 5., 0., 0.]])


In [59]:
s1=[[0,0,3,4],[2,1,3,4]]
set1=torch.tensor(s1,dtype=torch.float)
cxcy=xy_to_cxcy(set1)
print("Cx_C_y",cxcy)
print("xy",cxcy_to_xy(cxcy))
gcxgcy=cxcy_to_gcxgcy(cxcy,set1)
print("G_cx_G_cy",gcxgcy)

Cx_C_y tensor([[1.5000, 2.0000, 3.0000, 4.0000],
        [2.5000, 2.5000, 1.0000, 3.0000]])
xy tensor([[0., 0., 3., 4.],
        [2., 1., 3., 4.]])
G_cx_G_cy tensor([[ 5.0000,  5.0000,  0.0000,  0.0000],
        [ 1.6667,  3.7500, -5.4931, -1.4384]])


# Jaccard Overlap

In [0]:
def find_intersection(set_1, set_2):
    """
    Find the intersection of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: intersection of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # PyTorch auto-broadcasts singleton dimensions
    lower_bounds = torch.max(set_1[:, :2].unsqueeze(1), set_2[:, :2].unsqueeze(0))  # (n1, n2, 2)
    upper_bounds = torch.min(set_1[:, 2:].unsqueeze(1), set_2[:, 2:].unsqueeze(0))  # (n1, n2, 2)
    intersection_dims = torch.clamp(upper_bounds - lower_bounds, min=0)  # (n1, n2, 2)
    return intersection_dims[:, :, 0] * intersection_dims[:, :, 1]  # (n1, n2)


def find_jaccard_overlap(set_1, set_2):
    """
    Find the Jaccard Overlap (IoU) of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: Jaccard Overlap of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # Find intersections
    intersection = find_intersection(set_1, set_2)  # (n1, n2)

    # Find areas of each box in both sets
    areas_set_1 = (set_1[:, 2] - set_1[:, 0]) * (set_1[:, 3] - set_1[:, 1])  # (n1)
    areas_set_2 = (set_2[:, 2] - set_2[:, 0]) * (set_2[:, 3] - set_2[:, 1])  # (n2)

    # Find the union
    # PyTorch auto-broadcasts singleton dimensions
    union = areas_set_1.unsqueeze(1) + areas_set_2.unsqueeze(0) - intersection  # (n1, n2)

    return intersection / union  # (n1, n2)

In [61]:
s1=[[0,0,3,4],[0,0,3,4]]
set1=torch.tensor(s1,dtype=torch.float)
s2=[[2,0,5,4],[1,3,3,6],[1,0,2,2]]
set2=torch.tensor(s2,dtype=torch.float)
#find_jaccard_overlap(set1,set2)
print("Bounding boxes",set1)
print("Anchor Boxes",set2)

Bounding boxes tensor([[0., 0., 3., 4.],
        [0., 0., 3., 4.]])
Anchor Boxes tensor([[2., 0., 5., 4.],
        [1., 3., 3., 6.],
        [1., 0., 2., 2.]])


In [62]:
set1.shape

torch.Size([2, 4])

In [63]:
set1[:, :2].unsqueeze(dim=1)  #2x1x2

tensor([[[0., 0.]],

        [[0., 0.]]])

In [64]:
set2[:, :2].unsqueeze(dim=0) #1x3x2

tensor([[[2., 0.],
         [1., 3.],
         [1., 0.]]])

In [65]:
#lower_bound
torch.max(set1[:, :2].unsqueeze(1), set2[:, :2].unsqueeze(0))

tensor([[[2., 0.],
         [1., 3.],
         [1., 0.]],

        [[2., 0.],
         [1., 3.],
         [1., 0.]]])

In [66]:
#upper_bound
torch.min(set1[:, 2:].unsqueeze(1), set2[:, 2:].unsqueeze(0))

tensor([[[3., 4.],
         [3., 4.],
         [2., 2.]],

        [[3., 4.],
         [3., 4.],
         [2., 2.]]])

In [67]:
lower_bounds = torch.max(set1[:, :2].unsqueeze(1), set2[:, :2].unsqueeze(0))  # (n1, n2, 2)
upper_bounds = torch.min(set1[:, 2:].unsqueeze(1), set2[:, 2:].unsqueeze(0))  # (n1, n2, 2)
intersection_dims = torch.clamp(upper_bounds - lower_bounds, min=0)
print("Width and Height of Intersection",intersection_dims)

Width and Height of Intersection tensor([[[1., 4.],
         [2., 1.],
         [1., 2.]],

        [[1., 4.],
         [2., 1.],
         [1., 2.]]])


In [68]:
x=find_intersection(set1,set2)
print("Area of intersection",x)

Area of intersection tensor([[4., 2., 2.],
        [4., 2., 2.]])


In [69]:
areas_set_1 = (set1[:, 2] - set1[:, 0]) * (set1[:, 3] - set1[:, 1])  # (n1)
print("Area Set1 :",areas_set_1[0])
areas_set_2 = (set2[:, 2] - set2[:, 0]) * (set2[:, 3] - set2[:, 1])  # (n2)
print("Area Set2 :",areas_set_2)
union = areas_set_1.unsqueeze(1) + areas_set_2.unsqueeze(0) - x  # (n1, n2)
print("Union: ",union[0])

Area Set1 : tensor(12.)
Area Set2 : tensor([12.,  6.,  2.])
Union:  tensor([20., 16., 12.])


In [70]:
find_jaccard_overlap(set1,set2)

tensor([[0.2000, 0.1250, 0.1667],
        [0.2000, 0.1250, 0.1667]])

# Decimate

In [0]:
def decimate(tensor, m):
    """
    Decimate a tensor by a factor 'm', i.e. downsample by keeping every 'm'th value.

    This is used when we convert FC layers to equivalent Convolutional layers, BUT of a smaller size.

    :param tensor: tensor to be decimated
    :param m: list of decimation factors for each dimension of the tensor; None if not to be decimated along a dimension
    :return: decimated tensor
    """
    assert tensor.dim() == len(m)
    for d in range(tensor.dim()):
        if m[d] is not None:
            print("Index:",d)
            print("Number of elements in d = {} (dimension) : {}".format(d,tensor.size(d)))
            print("Step Size:",m[d])
            '''
            Returns a new tensor which indexes the input tensor along dimension dim using the entries in index which is a LongTensor.
            1st iteration start-0 - end-2 step_size-4   -selects just 1st element
            2nd iteration Ignore 
            3rd iteration start-0 - end-7 step_size-3   -selects  1st , 4th and 7th element of current axis
            4th iteration start-0 - end-2 step_size-3   -selects 1st , 4th and 7th element of  current axis
            '''
            #Keep subsetting across dimensions 
            tensor = tensor.index_select(dim=d,
                                         index=torch.arange(start=0, end=tensor.size(d), step=m[d]).long())
            print(tensor)

    return tensor

  

In [72]:
s1=torch.randn([2,2,7,7])
print("Dimension of s1:", s1.dim())
m=[4,None,3,3]  #Number of filters (Divide by 4(4096/4)),None-Input Channels,
print(s1)

Dimension of s1: 4
tensor([[[[-0.2546, -1.5330,  0.9464,  0.8398, -2.0292,  0.2078, -0.1742],
          [-0.9466, -0.3097, -1.5996,  1.0044,  0.1724,  0.1705, -0.7411],
          [-1.2677,  0.3842,  1.6150, -0.9137, -1.1277,  1.9705, -0.5900],
          [ 1.3535,  0.1026, -1.4232, -0.2712, -0.6568, -2.1320,  1.2154],
          [-0.9880, -1.6161, -0.2654,  0.1858,  0.5914,  0.2180, -1.3067],
          [ 0.0613, -2.4601, -0.7081, -1.0835, -1.0823,  0.4095, -2.2745],
          [ 0.7307,  0.0312, -0.6315, -0.8049, -0.0664,  0.3020,  0.3811]],

         [[ 0.0514, -0.9905, -1.2673, -0.3777,  1.1829, -1.0737,  0.6938],
          [-0.4404,  0.0302,  1.1418, -0.1971, -0.5233,  0.0451,  0.8625],
          [-0.4771, -1.4032,  0.4864, -1.5147, -0.5819,  0.5870, -1.3267],
          [ 1.3249,  0.1047,  0.4748,  0.2062, -0.3078,  0.3360,  1.1187],
          [ 0.1928, -0.8406, -2.8185, -0.0976,  0.5295, -1.6954, -0.4845],
          [-0.9378,  1.2767, -0.9744, -0.3748,  1.0450, -0.9261,  0.5624],
    

In [73]:
x=decimate(s1,m)

Index: 0
Number of elements in d = 0 (dimension) : 2
Step Size: 4
tensor([[[[-0.2546, -1.5330,  0.9464,  0.8398, -2.0292,  0.2078, -0.1742],
          [-0.9466, -0.3097, -1.5996,  1.0044,  0.1724,  0.1705, -0.7411],
          [-1.2677,  0.3842,  1.6150, -0.9137, -1.1277,  1.9705, -0.5900],
          [ 1.3535,  0.1026, -1.4232, -0.2712, -0.6568, -2.1320,  1.2154],
          [-0.9880, -1.6161, -0.2654,  0.1858,  0.5914,  0.2180, -1.3067],
          [ 0.0613, -2.4601, -0.7081, -1.0835, -1.0823,  0.4095, -2.2745],
          [ 0.7307,  0.0312, -0.6315, -0.8049, -0.0664,  0.3020,  0.3811]],

         [[ 0.0514, -0.9905, -1.2673, -0.3777,  1.1829, -1.0737,  0.6938],
          [-0.4404,  0.0302,  1.1418, -0.1971, -0.5233,  0.0451,  0.8625],
          [-0.4771, -1.4032,  0.4864, -1.5147, -0.5819,  0.5870, -1.3267],
          [ 1.3249,  0.1047,  0.4748,  0.2062, -0.3078,  0.3360,  1.1187],
          [ 0.1928, -0.8406, -2.8185, -0.0976,  0.5295, -1.6954, -0.4845],
          [-0.9378,  1.2767, -0.

In [74]:
x.shape

torch.Size([1, 2, 3, 3])

# Create Prior Boxes


In [0]:
def create_prior_boxes(self):
        """
        Create the 8732 prior (default) boxes for the SSD300, as defined in the paper.

        :return: prior boxes in center-size coordinates, a tensor of dimensions (8732, 4)
        """
        fmap_dims = {'conv4_3': 38,
                     'conv7': 19,
                     'conv8_2': 10,
                     'conv9_2': 5,
                     'conv10_2': 3,
                     'conv11_2': 1}

        obj_scales = {'conv4_3': 0.1,
                      'conv7': 0.2,
                      'conv8_2': 0.375,
                      'conv9_2': 0.55,
                      'conv10_2': 0.725,
                      'conv11_2': 0.9}

        aspect_ratios = {'conv4_3': [1., 2., 0.5],
                         'conv7': [1., 2., 3., 0.5, .333],
                         'conv8_2': [1., 2., 3., 0.5, .333],
                         'conv9_2': [1., 2., 3., 0.5, .333],
                         'conv10_2': [1., 2., 0.5],
                         'conv11_2': [1., 2., 0.5]}

        fmaps = list(fmap_dims.keys())

        prior_boxes = []

        for k, fmap in enumerate(fmaps):
            for i in range(fmap_dims[fmap]):
                for j in range(fmap_dims[fmap]):
                    cx = (j + 0.5) / fmap_dims[fmap]
                    cy = (i + 0.5) / fmap_dims[fmap]

                    for ratio in aspect_ratios[fmap]:
                        prior_boxes.append([cx, cy, obj_scales[fmap] * sqrt(ratio), obj_scales[fmap] / sqrt(ratio)])

                        # For an aspect ratio of 1, use an additional prior whose scale is the geometric mean of the
                        # scale of the current feature map and the scale of the next feature map
                        if ratio == 1.:
                            try:
                                additional_scale = sqrt(obj_scales[fmap] * obj_scales[fmaps[k + 1]])
                            # For the last feature map, there is no "next" feature map
                            except IndexError:
                                additional_scale = 1.
                            prior_boxes.append([cx, cy, additional_scale, additional_scale])

        prior_boxes = torch.FloatTensor(prior_boxes).to(device)  # (8732, 4)
        prior_boxes.clamp_(0, 1)  # (8732, 4)

        return prior_boxes

In [0]:
def create_prior_boxes():
        """
        Create the 8732 prior (default) boxes for the SSD300, as defined in the paper.

        :return: prior boxes in center-size coordinates, a tensor of dimensions (8732, 4)
        """
        fmap_dims = {'conv10_2': 3,
                    'conv11_2': 1}

        obj_scales = {'conv10_2': 0.725,
                     'conv11_2': 0.9}

        aspect_ratios = {'conv10_2': [1., 2., 0.5],
                        'conv11_2': [1., 2., 0.5]}

        fmaps = list(fmap_dims.keys())
        prior_boxes = []

        for k, fmap in enumerate(fmaps):
            print("For feature map = {}".format(fmap))
            for i in range(fmap_dims[fmap]):
                #print(i)
                #print("y location on Feature Map(i) = {} and fmap = {}".format(i,fmap))
                for j in range(fmap_dims[fmap]):
                    #print(j)
                    print("x,y location on Feature Map(j,i) = ({},{})".format(j,i))
                    cx = (j + 0.5) / fmap_dims[fmap]
                    cy = (i + 0.5) / fmap_dims[fmap]
                    print("cx = {} and cy = {}".format(cx,cy))

                    for m,ratio in enumerate(aspect_ratios[fmap]):
                        prior_boxes.append([cx, cy, obj_scales[fmap] * sqrt(ratio), obj_scales[fmap] / sqrt(ratio)])
                        print("For aspect ration {}".format(ratio))
                        print(prior_boxes)
                        #print("Pre\n Centre of x ={}\n Centre of y = {}\n Widht ={}\n Height {}".format(prior_boxes[m][0],prior_boxes[m][1],prior_boxes[m][2],prior_boxes[m][3]))
                        # For an aspect ratio of 1, use an additional prior whose scale is the geometric mean of the
                        # scale of the current feature map and the scale of the next feature map
                        if ratio == 1.:
                            print("Add 1 more prior box")
                            try:
                                additional_scale = sqrt(obj_scales[fmap] * obj_scales[fmaps[k + 1]])
                                print("Additional Scale",additional_scale)
                            # For the last feature map, there is no "next" feature map
                            except IndexError:
                                additional_scale = 1.
                            prior_boxes.append([cx, cy, additional_scale, additional_scale])
                            print(prior_boxes)
                            #print(" Post \n Modified Centre of x ={}\n Centre of y = {}\n Widht ={}\n Height {}".format(prior_boxes[m+1][0],prior_boxes[i][1],prior_boxes[i][2],prior_boxes[i][3]))
        prior_boxes = torch.FloatTensor(prior_boxes).to(device)  # (8732, 4)
        prior_boxes.clamp_(0, 1)  # (8732, 4)

        return prior_boxes

In [77]:
print("After clamping values between 0 and 1\n",create_prior_boxes())

For feature map = conv10_2
x,y location on Feature Map(j,i) = (0,0)
cx = 0.16666666666666666 and cy = 0.16666666666666666
For aspect ration 1.0
[[0.16666666666666666, 0.16666666666666666, 0.725, 0.725]]
Add 1 more prior box
Additional Scale 0.8077747210701756
[[0.16666666666666666, 0.16666666666666666, 0.725, 0.725], [0.16666666666666666, 0.16666666666666666, 0.8077747210701756, 0.8077747210701756]]
For aspect ration 2.0
[[0.16666666666666666, 0.16666666666666666, 0.725, 0.725], [0.16666666666666666, 0.16666666666666666, 0.8077747210701756, 0.8077747210701756], [0.16666666666666666, 0.16666666666666666, 1.0253048327204939, 0.5126524163602469]]
For aspect ration 0.5
[[0.16666666666666666, 0.16666666666666666, 0.725, 0.725], [0.16666666666666666, 0.16666666666666666, 0.8077747210701756, 0.8077747210701756], [0.16666666666666666, 0.16666666666666666, 1.0253048327204939, 0.5126524163602469], [0.16666666666666666, 0.16666666666666666, 0.5126524163602469, 1.0253048327204939]]
x,y location on

# MultiLoss

What we have:
1. Two  Bounding Boxes corresponding to two objects
2. 3 anchor boxes 
3. True Labels for each bounding boxes

What we require:
1. Labels for each prior boxes 

Steps:
1. Assign true labels to anchor boxes
<br>
        For each bounding box :
         a. Calculate the overlap with the anchor boxes -Jaccard Index
         b. Identify the anchor box with the highest overlap.
         c.  Assign the anchor box with the true lable of the bounding box.
2. Calculate the Localization Loss(Done only for positive priors)
<br>
       Use the mask tensor to calculate loss over the relevant anchor boxes offsets.
3. Hard Negative Mining
4. Calculate the Confidence Loss


In [78]:
boxes=[[0,0,3,4],[0,0,3,6]]
boxes=torch.tensor(boxes,dtype=torch.float)
priors=[[0,0,5,4],[0,2.8,3,6],[0,0,4,7],[2.8,2.8,3,6]]
priors=torch.tensor(priors,dtype=torch.float)
#find_jaccard_overlap(set1,set2)
print("2 Boundin boxes\n",boxes)
print(" 3 Anchor Boxes\n",priors)

2 Boundin boxes
 tensor([[0., 0., 3., 4.],
        [0., 0., 3., 6.]])
 3 Anchor Boxes
 tensor([[0.0000, 0.0000, 5.0000, 4.0000],
        [0.0000, 2.8000, 3.0000, 6.0000],
        [0.0000, 0.0000, 4.0000, 7.0000],
        [2.8000, 2.8000, 3.0000, 6.0000]])


In [79]:
priors_cxcy=xy_to_cxcy(priors)
print(priors_cxcy)

tensor([[2.5000, 2.0000, 5.0000, 4.0000],
        [1.5000, 4.4000, 3.0000, 3.2000],
        [2.0000, 3.5000, 4.0000, 7.0000],
        [2.9000, 4.4000, 0.2000, 3.2000]])


## Step1:
1. Assign true labels to anchor boxes
<br>
        For each bounding box :
         a. Calculate the overlap with the anchor boxes -Jaccard Index
         b. Identify the anchor box with the highest overlap by making it 1 and the remaining to 0(masking)  if overlap is less tha 0.5 .
         c.  Assign the anchor box with the true lable of the bounding box.


In [80]:
#Bounding box = 2
labels=torch.tensor([[2],[3]])
n_objects=boxes.size(0)
overlap=find_jaccard_overlap(boxes,priors)
overlap   #2 x 4
#Row is the object
#COlumn is the priors
print("Overlap between Bounding Box and Anchor Boxes\n",overlap)
#Desired Outcome -1. Assign lable of 1st Bounding box to 1st anchor box
                  #2. Assign label of 2nd Bounding box to 3rd anchor box

Overlap between Bounding Box and Anchor Boxes
 tensor([[0.6000, 0.2000, 0.4286, 0.0194],
        [0.4615, 0.5333, 0.6429, 0.0356]])


For each prior, find the object that has the maximum overlap

In [81]:
overlap_for_each_prior, object_for_each_prior=overlap.max(dim=0)
print(overlap.max(dim=0))   #4

(tensor([0.6000, 0.5333, 0.6429, 0.0356]), tensor([0, 1, 1, 1]))


For each object.find the prior that has the maximum overlap 

In [82]:
_, prior_for_each_object = overlap.max(dim=1)
print(overlap.max(dim=1))  #2

(tensor([0.6000, 0.6429]), tensor([0, 2]))


For each priors, assign each object to the corresponding maximum-overlap-prior. (This fixes 1.)
            

In [83]:
object_for_each_prior[prior_for_each_object] = torch.LongTensor(range(n_objects)).to(device)
print("Identified object for each prior",object_for_each_prior)

Identified object for each prior tensor([0, 1, 1, 1])


In [84]:
overlap_for_each_prior[prior_for_each_object] = 1.
print("Overlap of Identified object for each prior", overlap_for_each_prior)

Overlap of Identified object for each prior tensor([1.0000, 0.5333, 1.0000, 0.0356])


In [85]:
# Encode center-size object coordinates into the form we regressed predicted boxes to
true_locs = cxcy_to_gcxgcy(xy_to_cxcy(set1[object_for_each_prior]), priors_cxcy)  # (8732, 4)
print(true_locs)

tensor([[ -2.0000,   0.0000,  -2.5541,   0.0000],
        [  0.0000,  -7.5000,   0.0000,   1.1157],
        [ -1.2500,  -2.1429,  -1.4384,  -2.7981],
        [-70.0000,  -7.5000,  13.5402,   1.1157]])


In [0]:
threshold=0.5
# Labels for each prior
label_for_each_prior = labels[object_for_each_prior]  # (3)
# Set priors whose overlaps with objects are less than the threshold to be background (no object)
label_for_each_prior[overlap_for_each_prior < threshold] = 0  # (8732)

# Store
true_classes = label_for_each_prior

In [87]:
label_for_each_prior

tensor([[2],
        [3],
        [3],
        [0]])

In [88]:
positive_priors = true_classes != 0  # (N, 8732)
print("Mask tensor",positive_priors)

Mask tensor tensor([[1],
        [1],
        [1],
        [0]], dtype=torch.uint8)


Step 3: Hard Negative Mining



In [89]:
positive_priors.sum(dim=1)

tensor([1, 1, 1, 0])

In [90]:
neg_pos_ratio=3
n_positives = positive_priors.sum(dim=0)  # (N)
n_hard_negatives = neg_pos_ratio * n_positives  # (N)
print(n_positives,n_hard_negatives)

tensor([3]) tensor([9])
